In [1]:
# Project Overview: Create Relevant Spotify Playlist with Python
# Pre-work: Set up your Account & Register your application @ https://developer.spotify.com/documentation/web-api/quick-start/
# pip install requirements.txt
# 0. Declare name of playlist to be created
# 1. Import Packages and establish creds to connect to Spotify
# 2. Get raw Spotify data of Liked (Hearted) songs into a dataframe
# 3. Add audio features
# 4. Add genre data
# 5. Create genre score
### Repeat process (steps 2-5) for disliked songs (based on dislike playlist) ###
# 6. Create playlist df (to access disliked playlist)
# 7. Combine liked and disliked songs into my_songs dataframe with "Liked" flag
# 8. Get featured playlist uris so we can get their associated songs
# 9. Get raw data from the songs
# ....
# 10. Create playlist

In [2]:
# 0. Declare name of playlist to be created
from datetime import datetime, timedelta
pred_like_playlist_name = f"FreshJams @{str(datetime.now())}"

In [3]:
#### ALL FUNCTIONS BELOW ####

# 1. Import Packages and establish creds to connect to Spotify
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyOAuth

from config import CONFIG
import inspect
import io
import json
from IPython.display import display
from json.decoder import JSONDecodeError
import math
import os
import pandas as pd
import numpy as np 
from pandas.io.json import json_normalize
import pickle
import requests
import subprocess
import sys
import time
import webbrowser

API_BASE = 'https://accounts.spotify.com'
CLI_ID = CONFIG["env_variables"]['SPOTIPY_CLIENT_ID']
CLI_SEC = CONFIG["env_variables"]['SPOTIPY_CLIENT_SECRET']
REDIRECT_URI = CONFIG["env_variables"]['SPOTIPY_REDIRECT_URI']
SHOW_DIALOG = False
SCOPE = '''
    playlist-modify-private, 
    playlist-modify-public,
    playlist-read-collaborative,
    playlist-read-private,
    user-follow-modify,
    user-follow-read,
    user-library-modify,
    user-library-read,
    user-modify-playback-state, 
    user-read-currently-playing, 
    user-read-email,
    user-read-playback-state, 
    user-read-private,
    user-read-recently-played, 
    user-top-read'''

def req(url,headers):
    r = requests.get(url, headers=headers)
    return json.loads(r.text)

#1. get raw data for liked songs (i.e. hearted)
def get_raw_liked_song_list():
    print('Get_raw_liked_song_list')
    offset = 0
    url = f"https://api.spotify.com/v1/me/tracks?limit=50&offset={offset}"

    liked_song_count = req(url,headers=headers)["total"]
    print(f"Liked song count: {liked_song_count}")

    raw_liked_songs = []
    for i in range(int(math.ceil(liked_song_count/50.0))):
        offset = 50*i
        raw_liked_songs.extend(req(f"https://api.spotify.com/v1/me/tracks?limit=50&offset={offset}",headers=headers)["items"])

    print("Example of song raw data from spotify:")
    print(f"{raw_liked_songs[0]}")
    return raw_liked_songs

# 2b. transform liked song raw data list iddnto df
# parse out data from Liked Songs' raw data
def song_metadata_to_df(raw_songs):
    print('Transforming song raw data list into DataFrame(df) - this is basically a python version of an excel spreadsheet')
    define_scope()
    # create empty df
    df = pd.DataFrame(columns = [
        'album', 
        'artist_uri', 
        'artist',
        'track',
        'url',
        'popularity',
        'added_at',
        'release_date',
        'release_year',
        'uri'
        ])
    # parse out songs 100 at a time
    for i in range(0, len(raw_songs)):
        if (i/100).is_integer():
            print(f"{i}/{len(raw_songs)} parsed")
        try:
            album = raw_songs[i]['track']['album']['name']
            artist_uri = raw_songs[i]['track']['artists'][0]['uri'].split(':')[2]
            artist = raw_songs[i]['track']['album']['artists'][0]['name']
            track = raw_songs[i]['track']['name']
            url = raw_songs[i]['track']['external_urls']['spotify']
            popularity = raw_songs[i]['track']['popularity']
            added_at = raw_songs[i]['added_at']
            release_date = raw_songs[i]['track']['album']['release_date']
            release_year = release_date.split('-')[0]
            uri = raw_songs[i]['track']['uri']
            df = df.append(pd.DataFrame([[
                album, 
                artist_uri,
                artist, 
                track, 
                url, 
                popularity, 
                added_at, 
                release_date, 
                release_year,
                uri
                ]], columns = [
                'album', 
                'artist_uri', 
                'artist',
                'track',
                'url',
                'popularity',
                'added_at',
                'release_date',
                'release_year',
                'uri'
                ]))
        except:
            pass
    df.set_index('uri', inplace=True)
    display(df.head())
    return df

#3. Add audio features 
# Combine liked song df with audio feature df
def add_audio_feats(df):
    print('Combining song_df with audio_feature_df')
    # get list of song uris
    song_uri_list = df.index.to_list()
    audio_feature_df = pd.DataFrame()
    for i in range(0,math.ceil(len(song_uri_list)/100)):
        features = sp.audio_features(song_uri_list[(i)*100:100*(i+1)])
        audio_feature_df = audio_feature_df.append(pd.DataFrame(features))
    audio_feature_df.set_index('uri', inplace=True)
    # merge this 
    liked_song_df = df.merge(audio_feature_df, on = 'uri')
    display(liked_song_df.head())
    return liked_song_df.drop_duplicates()

#4. Add genre data
# 4a. Get artist_uris -> get genres
def get_genres(df):
    print('Getting genres ...')
    g = []
    artist_uris = df['artist_uri'].unique()
    print(f"{len(artist_uris)} distinct artists")
    for i in range(0, len(artist_uris)):
        if (i/100).is_integer():
            print(f"{i}/{len(artist_uris)} parsed")
        request = req(url = f"https://api.spotify.com/v1/artists/{artist_uris[i]}",headers=headers)
        artist = request['name']
        genres = request['genres']
        print(f"{i}:{artist}:{genres}")
        g.append(genres)
#         socketio.sleep(0.1)

    genres_df = pd.DataFrame({'artist_uri': artist_uris, 'genre list': g})
    display(genres_df.head())
    return genres_df

# There is a list of genres for one artist that must be broken out
def explode_genres(df):
    genre_exploded_df = df.explode('genre list')
    
    print('genre_exploded_df')
    display(genre_exploded_df.head())
    return genre_exploded_df

# Add these genres back to the df for each song
def add_genres(df, genre_exploded_df):
    # the following does not work oddly enough
    # liked_song_df.reset_index().merge(genre_exploded_df, on = 'artist_uri', how = 'left').set_index('uri', inplace = True)
    df = df.reset_index().merge(genre_exploded_df, on = 'artist_uri', how = 'left').set_index('uri')
    df.rename(columns={'genre list': 'genre'}, inplace=True)
    
    print('Merging genres into song_df')
    display(df.head())
    return df

#5. Create Genre Score
# Get genre counts for liked songs - the higher the count, the higher the score
def get_genre_counts(genre_exploded_df):
    print('Getting genre counts')
    # establish genre counts (which will determine score)
    genre_count_df = pd.DataFrame(genre_exploded_df['genre list'].value_counts()).reset_index()
    genre_count_df.columns = ['genre','genre count']
    print('genre_count_df.head()')
    display(genre_count_df.head())
    return genre_count_df

# Merge the genre counts with the song df and sum the genre column
def establish_genre_score(genre_count_df, df):
    print('Merging genre counts with song_df')
    df = df.reset_index().merge(genre_count_df , on = 'genre',how = 'left')
    df.drop(columns=['genre'], inplace=True)
    display(df.head())

    groupby_cols = df.columns.to_list()[:-1]
    
    print('Summing genre column')
    df = df.groupby(groupby_cols).sum().sort_values('genre count', ascending=False)
    df = df.rename({'genre count': 'genre score'}, axis=1).reset_index() 
    # reset index after summing - otherwise everything prior to sum will become index
    df.set_index('uri', inplace=True)
    display(df.head())
    return df

# 6. Create playlist df (to access disliked playlist)
# get playlist raw data (for dislikes)
def get_playlist_uris(playlists_raw):
    df = pd.DataFrame()
    for i in range(0, len(playlists_raw)):
        try:
            playlist = playlists_raw[i]['name']
        except:
            playlist = ''
        try:
            playlist_uri = playlists_raw[i]['uri']
        except:
            playlist_uri = ''
        try:
            song_count = playlists_raw[i]['tracks']['total']
        except:
            song_count = 0
        df = df.append(pd.DataFrame([[playlist, playlist_uri, song_count]], columns = ['playlist','playlist_uri','song_count']))
    df.set_index(['playlist'], inplace=True)
    display(df)
    return df

def get_raw_disliked_song_list():
    playlists_raw = sp.current_user_playlists()['items']
    playlists_df = get_playlist_uris(playlists_raw)
    display(playlists_df)
    uri = playlists_df.loc['Dislikes']['playlist_uri']
    song_count = playlists_df.loc['Dislikes']['song_count']
    # extract data from Disliked songs
    dislike_raw_list = []
    for i in range(int(math.ceil(song_count/100.0))):
        dislike_raw_list.extend(sp.user_playlist_tracks(uri.split(':')[1], uri.split(':')[2], offset=i*100)['items'])
    print('Getting raw disliked song list')
    return dislike_raw_list

# 7. Combine liked and disliked songs into my_songs dataframe with "Liked" flag
def combine_dfs_to_create_my_songs_df(df1,df2, save_destination):
    print('Combining dfs to create my_songs_df')
    df1['Liked'] = 1
    df2['Liked'] = 0
    my_songs_df = pd.concat([df1,df2])
    #my_songs_df.to_:w(f"{save_destination}/my_songs.csv")
    #my_songs_df.to_pickle(f"{save_destination}/my_songs.pkl")
    display(my_songs_df.head())
    #print('successfully saved to .csv and .pkl files in save_destination')
    return 'OK'

# 8. Get featured playlist uris so we can get their associated songs
def get_featured_playlist_uris(country, timestamp):
    print('Getting raw_featured_playlist_df')
    url = f"https://api.spotify.com/v1/browse/featured-playlists?limit=50&offset=0&country={country}&timestamp={timestamp}"
    featured_playlist_count = req(url,headers=headers)['playlists']['total']
    df = pd.DataFrame()
    for i in range(0,featured_playlist_count):
        raw_featured_playlist = req(url,headers=headers)['playlists']['items']
        playlist = raw_featured_playlist[i]['name']
        playlist_uri = raw_featured_playlist[i]['uri']
        song_count = raw_featured_playlist[i]['tracks']['total']
        df = df.append(pd.DataFrame([[playlist, playlist_uri, song_count]], columns = ['playlist','playlist_uri','song_count']))
    df.set_index(['playlist'], inplace=True)
    display(df)
    return df

# 9. Get raw data from the songs
def get_raw_featured_playlist_song_list(df):
    raw_featured_songs = []
    for uri in df.loc[:,'playlist_uri'].to_list():
        song_count = df.loc[df['playlist_uri']==uri,'song_count'][0]
        offset = 0
        url = f"https://api.spotify.com/v1/playlists/{uri.split(':')[2]}/tracks?limit=50&offset={offset}"
        for i in range(int(math.ceil(song_count/100.0))):
            offset = 100*i
            raw_featured_songs.extend(req(url,headers=headers)['items'])

    return raw_featured_songs

def create_playlist(pred_like_playlist_name, df):
    # create playlist
    define_scope()
    payload = {"name": pred_like_playlist_name}
    result = requests.post(url = f'https://api.spotify.com/v1/users/{username}/playlists'
                    , headers = headers
                    , data = json.dumps(payload)
                    )
    playlists_raw = sp.current_user_playlists()['items']
    playlists_df = get_playlist_uris(playlists_raw)
    playlist_uri = get_playlist_uris(playlists_raw).loc[playlists_df.index == pred_like_playlist_name,'playlist_uri'][0].split(':')[2]
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rank.html
    df['pct_rank'] = df['genre score'].rank(pct=True)
    df = df.drop_duplicates()
    display(df.head())
    # print((1-(1-1/((int(genre_score_threshold)/50)+1.1))))
    uri_list = df.loc[df['pct_rank'] >= 0.95].reset_index()['uri'].to_list()
    num_songs_to_add = len(uri_list)
    # add songs
    for i in range(int(math.ceil(num_songs_to_add/100.0))):
        url = f"https://api.spotify.com/v1/playlists/{playlist_uri}/tracks?uris={','.join(uri_list[i*100:(i+1)*100])}"
        requests.post(url, headers=headers)
    return 'OK'

def get_my_top_artists():
    my_top_artists = req(url = 'https://api.spotify.com/v1/me/top/artists?limit=50',headers=headers)['items']
    my_top_artists_df = pd.DataFrame(columns=['artist_uri','name','genre list'])

    for i in range(len(my_top_artists)):
        uri = my_top_artists[i]['uri']
        name = my_top_artists[i]['name']
        genres = my_top_artists[i]['genres']

        my_top_artists_df = my_top_artists_df.append(pd.DataFrame([[uri, name, genres]], columns = ['artist_uri','name', 'genre list']))
    my_top_artists_exploded_genres_df = explode_genres(my_top_artists_df)
    display(my_top_artists_exploded_genres_df.head())
    return get_genre_counts(my_top_artists_exploded_genres_df)

# def verify():
#     global auth_url
#     auth_url = f'{API_BASE}/authorize?client_id={CLI_ID}&response_type=code&redirect_uri={REDIRECT_URI}&scope={SCOPE}&show_dialog={SHOW_DIALOG}'
#     return auth_url

# def api_callback():
#     global token
# #     session.clear()
#     code = request.args.get('code')

#     auth_token_url = f"{API_BASE}/api/token"
#     res = requests.post(auth_token_url, data={
#         "grant_type":"authorization_code",
#         "code":code,
#         "redirect_uri":REDIRECT_URI,
#         "client_id":CLI_ID,
#         "client_secret":CLI_SEC
#         })

#     res_body = res.json()
#     print(res.json())
#     token = res_body.get("access_token")
#     return token

def define_scope():
    global scope, sp, user, username, headers
    scope = '''
    playlist-modify-private 
    playlist-modify-public 
    playlist-read-collaborative 
    playlist-read-private
    user-follow-modify 
    user-follow-read 
    user-library-modify 
    user-library-read 
    user-modify-playback-state 
    user-read-currently-playing 
    user-read-email 
    user-read-playback-state 
    user-read-private 
    user-read-recently-played 
    user-top-read'''
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
    user = sp.current_user()
    username = user['display_name']
    token = util.prompt_for_user_token(username, scope)
    headers = {'Authorization': "Bearer {}".format(token)}
    print(f"Logged in as {user['display_name']}")
    return

/Users/FYE7200/Documents/Real_Time_Products/repos/spotipy/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  CONFIG = yaml.load(yaml_file)
/Users/FYE7200/.virtualenvs/spotipy/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [4]:
define_scope()
print(f"Your new playlist: {pred_like_playlist_name}")
# print(f"genre threshold = {genre_threshold}")
# print(f"featured playlist year = {featured_playlist_year} ({year_delta} years ago)")

print('Scope defined')
print('')
print('''****PART 1 of 5: Your "Spotify" Top Artists****''')
my_top_artists_genre_count_df = get_my_top_artists()

print('****PART 2 of 5: Your "Liked Songs" Artists (if you have liked (i.e. hearted) songs)****')
try:
    raw_liked_song_list = get_raw_liked_song_list()
    liked_song_df = song_metadata_to_df(raw_liked_song_list)
    liked_song_df = add_audio_feats(liked_song_df)
    genre_df = get_genres(liked_song_df)
    genre_exploded_df = explode_genres(genre_df)
    liked_song_df = add_genres(liked_song_df, genre_exploded_df)
    liked_genre_count_df = get_genre_counts(genre_exploded_df)
    liked_song_df = establish_genre_score(liked_genre_count_df, liked_song_df)
except:
    pass

Logged in as 1254636534
Your new playlist: FreshJams @2020-09-27 20:40:02.535628
Scope defined

****PART 1 of 5: Your "Spotify" Top Artists****
genre_exploded_df


,artist_uri,name,genre list
0,spotify:artist:1l9d7B8W0IHy3LqWsxP2SH,Phantogram,alternative dance
0,spotify:artist:1l9d7B8W0IHy3LqWsxP2SH,Phantogram,chillwave
0,spotify:artist:1l9d7B8W0IHy3LqWsxP2SH,Phantogram,electropop
0,spotify:artist:1l9d7B8W0IHy3LqWsxP2SH,Phantogram,indie pop
0,spotify:artist:1l9d7B8W0IHy3LqWsxP2SH,Phantogram,indie rock


,artist_uri,name,genre list
0,spotify:artist:1l9d7B8W0IHy3LqWsxP2SH,Phantogram,alternative dance
0,spotify:artist:1l9d7B8W0IHy3LqWsxP2SH,Phantogram,chillwave
0,spotify:artist:1l9d7B8W0IHy3LqWsxP2SH,Phantogram,electropop
0,spotify:artist:1l9d7B8W0IHy3LqWsxP2SH,Phantogram,indie pop
0,spotify:artist:1l9d7B8W0IHy3LqWsxP2SH,Phantogram,indie rock


Getting genre counts
genre_count_df.head()


,genre,genre count
0,indie folk,22
1,stomp and holler,20
2,indie pop,18
3,indie rock,16
4,new americana,14


...

,genre,genre count
0,indie pop,75
1,indie rock,74
2,stomp and holler,73
3,indie folk,63
4,new americana,52


Merging genre counts with song_df


,uri,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre count
0,spotify:track:7D56acHvA76RlIuebqxWKD,Moments / Tides,7o96HO2zrujyATtVsqGhh3,Goth Babe,Moments / Tides,https://open.spotify.com/track/7D56acHvA76RlIu...,62,2020-09-27T14:55:22Z,2020-08-05,2020,...,0.106,0.531,85.018,audio_features,7D56acHvA76RlIuebqxWKD,https://api.spotify.com/v1/tracks/7D56acHvA76R...,https://api.spotify.com/v1/audio-analysis/7D56...,204127,4,6.0
1,spotify:track:7D56acHvA76RlIuebqxWKD,Moments / Tides,7o96HO2zrujyATtVsqGhh3,Goth Babe,Moments / Tides,https://open.spotify.com/track/7D56acHvA76RlIu...,62,2020-09-27T14:55:22Z,2020-08-05,2020,...,0.106,0.531,85.018,audio_features,7D56acHvA76RlIuebqxWKD,https://api.spotify.com/v1/tracks/7D56acHvA76R...,https://api.spotify.com/v1/audio-analysis/7D56...,204127,4,75.0
2,spotify:track:7D56acHvA76RlIuebqxWKD,Moments / Tides,7o96HO2zrujyATtVsqGhh3,Goth Babe,Moments / Tides,https://open.spotify.com/track/7D56acHvA76RlIu...,62,2020-09-27T14:55:22Z,2020-08-05,2020,...,0.106,0.531,85.018,audio_features,7D56acHvA76RlIuebqxWKD,https://api.spotify.com/v1/tracks/7D56acHvA76R...,https://api.spotify.com/v1/audio-analysis/7D56...,204127,4,4.0
3,spotify:track:7D56acHvA76RlIuebqxWKD,Moments / Tides,7o96HO2zrujyATtVsqGhh3,Goth Babe,Moments / Tides,https://open.spotify.com/track/7D56acHvA76RlIu...,62,2020-09-27T14:55:22Z,2020-08-05,2020,...,0.106,0.531,85.018,audio_features,7D56acHvA76RlIuebqxWKD,https://api.spotify.com/v1/tracks/7D56acHvA76R...,https://api.spotify.com/v1/audio-analysis/7D56...,204127,4,74.0
4,spotify:track:7D56acHvA76RlIuebqxWKD,Moments / Tides,7o96HO2zrujyATtVsqGhh3,Goth Babe,Moments / Tides,https://open.spotify.com/track/7D56acHvA76RlIu...,62,2020-09-27T14:55:22Z,2020-08-05,2020,...,0.106,0.531,85.018,audio_features,7D56acHvA76RlIuebqxWKD,https://api.spotify.com/v1/tracks/7D56acHvA76R...,https://api.spotify.com/v1/audio-analysis/7D56...,204127,4,52.0


Summing genre column


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,danceability,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre score
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:0ey9VL7lFerteaCVvahKYV,Forgiveness Rock Record,7lOJ7WXyopaxri0dbOiZkd,Broken Social Scene,Sweetest Kill,https://open.spotify.com/track/0ey9VL7lFerteaC...,52,2020-03-12T22:40:03Z,2010-05-04,2010,0.709,...,0.0961,0.691,87.961,audio_features,0ey9VL7lFerteaCVvahKYV,https://api.spotify.com/v1/tracks/0ey9VL7lFert...,https://api.spotify.com/v1/audio-analysis/0ey9...,309307,4,511.0
spotify:track:03gKdAQAswBo7v4p8F1An4,You Forgot It In People,7lOJ7WXyopaxri0dbOiZkd,Broken Social Scene,Pacific Theme,https://open.spotify.com/track/03gKdAQAswBo7v4...,46,2020-06-13T21:26:33Z,2003-03-11,2003,0.723,...,0.0979,0.767,129.725,audio_features,03gKdAQAswBo7v4p8F1An4,https://api.spotify.com/v1/tracks/03gKdAQAswBo...,https://api.spotify.com/v1/audio-analysis/03gK...,309187,4,511.0
spotify:track:2NVaKeIAUZwN5tXnn12EMi,Fate,4mLJ3XfOM5FPjSAWdQ2Jk7,Dr. Dog,The Breeze,https://open.spotify.com/track/2NVaKeIAUZwN5tX...,58,2019-11-03T17:46:14Z,2008-07-22,2008,0.611,...,0.6120,0.319,90.124,audio_features,2NVaKeIAUZwN5tXnn12EMi,https://api.spotify.com/v1/tracks/2NVaKeIAUZwN...,https://api.spotify.com/v1/audio-analysis/2NVa...,239093,4,466.0
spotify:track:7ywp9MWERblgkSMoMBugC0,Holiday,6RWjTQqILL7a1tQ0VapyLK,The Magnetic Fields,Strange Powers,https://open.spotify.com/track/7ywp9MWERblgkSM...,50,2019-11-03T17:42:52Z,1994,1994,0.668,...,0.1290,0.963,131.767,audio_features,7ywp9MWERblgkSMoMBugC0,https://api.spotify.com/v1/tracks/7ywp9MWERblg...,https://api.spotify.com/v1/audio-analysis/7ywp...,161613,4,458.0
spotify:track:7LZN7FkxHZk6maiN6NdI2i,Sunlit Youth,75dQReiBOHN37fQgWQrIAJ,Local Natives,Dark Days,https://open.spotify.com/track/7LZN7FkxHZk6mai...,63,2020-03-14T18:53:56Z,2016-09-09,2016,0.604,...,0.1250,0.590,115.038,audio_features,7LZN7FkxHZk6maiN6NdI2i,https://api.spotify.com/v1/tracks/7LZN7FkxHZk6...,https://api.spotify.com/v1/audio-analysis/7LZN...,180413,4,457.0


In [5]:
print('''****PART 3 of 5: Your "Disliked" Artists (if you have a 'Dislikes' playlist)****''')
try:
    define_scope()
    raw_disliked_song_list = get_raw_disliked_song_list()
    disliked_song_df = song_metadata_to_df(raw_disliked_song_list)
    disliked_song_df = add_audio_feats(disliked_song_df)
    genre_df = get_genres(disliked_song_df)
    genre_exploded_df = explode_genres(genre_df)
    disliked_song_df = add_genres(disliked_song_df, genre_exploded_df)
    disliked_genre_count_df = get_genre_counts(genre_exploded_df)
    disliked_genre_count_df['genre count'] = disliked_genre_count_df['genre count'] * -1
    disliked_song_df = establish_genre_score(disliked_genre_count_df, disliked_song_df)
except:
    pass

****PART 3 of 5: Your "Disliked" Artists (if you have a 'Dislikes' playlist)****
Logged in as 1254636534


,playlist_uri,song_count
playlist,,
Jams 9/27 12:00,spotify:playlist:79EHB83wBhmDy5hdiess73,29
"Jams 9/26, 9:40pm",spotify:playlist:1g2dqbzbKfn4I7RfggDGHW,28
"Jams 9/26, 8:35pm",spotify:playlist:6WFIRi0vyvxn8ZGzndekEa,25
Dislikes,spotify:playlist:4zoDh60COnrbyctm9z9h77,454
My Shazam Tracks,spotify:playlist:5Gy5xebfLSaOms4klT0NAD,50
Discover Weekly,spotify:playlist:37i9dQZEVXcPcrlohfCkeD,30


,playlist_uri,song_count
playlist,,
Jams 9/27 12:00,spotify:playlist:79EHB83wBhmDy5hdiess73,29
"Jams 9/26, 9:40pm",spotify:playlist:1g2dqbzbKfn4I7RfggDGHW,28
"Jams 9/26, 8:35pm",spotify:playlist:6WFIRi0vyvxn8ZGzndekEa,25
Dislikes,spotify:playlist:4zoDh60COnrbyctm9z9h77,454
My Shazam Tracks,spotify:playlist:5Gy5xebfLSaOms4klT0NAD,50
Discover Weekly,spotify:playlist:37i9dQZEVXcPcrlohfCkeD,30


Getting raw disliked song list
Transforming song raw data list into DataFrame(df) - this is basically a python version of an excel spreadsheet
Logged in as 1254636534
0/454 parsed
100/454 parsed
200/454 parsed
300/454 parsed
400/454 parsed


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year
uri,,,,,,,,,
spotify:track:7nwnOXF8L52va1CKRPg1d4,Pink Friday: Roman Reloaded The Re-Up (Explici...,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Stupid Hoe,https://open.spotify.com/track/7nwnOXF8L52va1C...,39,2019-11-06T02:39:29Z,2012-01-01,2012
spotify:track:6muOWhMh7Tf0UFUtMDBhUR,Orphans / Arabesque,4gzpq5DPGxSnKTe4SA8HAU,Coldplay,Orphans,https://open.spotify.com/track/6muOWhMh7Tf0UFU...,67,2019-11-06T02:40:45Z,2019-10-24,2019
spotify:track:5jafMI8FLibnjkYTZ33m0c,The Bends,4Z8W4fKeB5YxbusRsdQVPb,Radiohead,High And Dry,https://open.spotify.com/track/5jafMI8FLibnjkY...,73,2019-11-06T02:45:04Z,1995-03-28,1995
spotify:track:04KJ4NGb2T22y26sTO41Q0,Rise Of An Empire (Deluxe Edition),5OrB6Jhhrl9y2PK0pSV4VP,Young Money,Moment,https://open.spotify.com/track/04KJ4NGb2T22y26...,39,2019-11-06T02:59:28Z,2014-01-01,2014
spotify:track:6sICylTFKeiplGHltWnJ1k,Z,43O3c6wewpzPKwVaGEEtBM,My Morning Jacket,What A Wonderful Man,https://open.spotify.com/track/6sICylTFKeiplGH...,33,2019-11-06T03:02:29Z,2005-10-04,2005


...

,genre,genre count
0,dance pop,45
1,pop,36
2,rock,34
3,pop rap,34
4,alternative rock,30


Merging genre counts with song_df


,uri,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre count
0,spotify:track:7nwnOXF8L52va1CKRPg1d4,Pink Friday: Roman Reloaded The Re-Up (Explici...,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Stupid Hoe,https://open.spotify.com/track/7nwnOXF8L52va1C...,39,2019-11-06T02:39:29Z,2012-01-01,2012,...,0.204,0.719,100.014,audio_features,7nwnOXF8L52va1CKRPg1d4,https://api.spotify.com/v1/tracks/7nwnOXF8L52v...,https://api.spotify.com/v1/audio-analysis/7nwn...,196893,4,-45.0
1,spotify:track:7nwnOXF8L52va1CKRPg1d4,Pink Friday: Roman Reloaded The Re-Up (Explici...,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Stupid Hoe,https://open.spotify.com/track/7nwnOXF8L52va1C...,39,2019-11-06T02:39:29Z,2012-01-01,2012,...,0.204,0.719,100.014,audio_features,7nwnOXF8L52va1CKRPg1d4,https://api.spotify.com/v1/tracks/7nwnOXF8L52v...,https://api.spotify.com/v1/audio-analysis/7nwn...,196893,4,-22.0
2,spotify:track:7nwnOXF8L52va1CKRPg1d4,Pink Friday: Roman Reloaded The Re-Up (Explici...,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Stupid Hoe,https://open.spotify.com/track/7nwnOXF8L52va1C...,39,2019-11-06T02:39:29Z,2012-01-01,2012,...,0.204,0.719,100.014,audio_features,7nwnOXF8L52va1CKRPg1d4,https://api.spotify.com/v1/tracks/7nwnOXF8L52v...,https://api.spotify.com/v1/audio-analysis/7nwn...,196893,4,-36.0
3,spotify:track:7nwnOXF8L52va1CKRPg1d4,Pink Friday: Roman Reloaded The Re-Up (Explici...,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Stupid Hoe,https://open.spotify.com/track/7nwnOXF8L52va1C...,39,2019-11-06T02:39:29Z,2012-01-01,2012,...,0.204,0.719,100.014,audio_features,7nwnOXF8L52va1CKRPg1d4,https://api.spotify.com/v1/tracks/7nwnOXF8L52v...,https://api.spotify.com/v1/audio-analysis/7nwn...,196893,4,-34.0
4,spotify:track:7nwnOXF8L52va1CKRPg1d4,Pink Friday: Roman Reloaded The Re-Up (Explici...,0hCNtLu0JehylgoiP8L4Gh,Nicki Minaj,Stupid Hoe,https://open.spotify.com/track/7nwnOXF8L52va1C...,39,2019-11-06T02:39:29Z,2012-01-01,2012,...,0.204,0.719,100.014,audio_features,7nwnOXF8L52va1CKRPg1d4,https://api.spotify.com/v1/tracks/7nwnOXF8L52v...,https://api.spotify.com/v1/audio-analysis/7nwn...,196893,4,-15.0


Summing genre column


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,danceability,...,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,genre score
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:00nQhTtc8FxgmB0NWypSE0,In Good Company,62W2NNKgzSx7GFJDUbJD6L,Java Gospel,You Rock My World,https://open.spotify.com/track/00nQhTtc8FxgmB0...,0,2020-04-02T13:32:30Z,2007-07-11,2007,0.662,...,0.3330,0.915,121.961,audio_features,00nQhTtc8FxgmB0NWypSE0,https://api.spotify.com/v1/tracks/00nQhTtc8Fxg...,https://api.spotify.com/v1/audio-analysis/00nQ...,245947,4,0.0
spotify:track:2x5C1qAOJ1KJFqM6uhOsgT,Blocked Numbers,2GdgDnUG0ZMFZiu2122pGQ,Crystal Skulls,Hussy,https://open.spotify.com/track/2x5C1qAOJ1KJFqM...,16,2020-06-29T13:48:04Z,2005-01-01,2005,0.660,...,0.1290,0.792,119.568,audio_features,2x5C1qAOJ1KJFqM6uhOsgT,https://api.spotify.com/v1/tracks/2x5C1qAOJ1KJ...,https://api.spotify.com/v1/audio-analysis/2x5C...,279093,4,0.0
spotify:track:5nB5bek72WORC5jegCIPNk,Untitled,2mxe0TnaNL039ysAj51xPQ,R. Kelly,Number One,https://open.spotify.com/track/5nB5bek72WORC5j...,40,2020-04-02T16:30:51Z,2009-11-27,2009,0.650,...,0.1800,0.576,136.796,audio_features,5nB5bek72WORC5jegCIPNk,https://api.spotify.com/v1/tracks/5nB5bek72WOR...,https://api.spotify.com/v1/audio-analysis/5nB5...,260867,4,0.0
spotify:track:6QeMRyd3BmSfJbaNay3dpe,R&B - 100 Hits - The Greatest R n B album - 10...,2mxe0TnaNL039ysAj51xPQ,Various Artists,The World's Greatest,https://open.spotify.com/track/6QeMRyd3BmSfJba...,53,2020-04-25T23:19:22Z,2013-06-21,2013,0.661,...,0.0469,0.721,95.971,audio_features,6QeMRyd3BmSfJbaNay3dpe,https://api.spotify.com/v1/tracks/6QeMRyd3BmSf...,https://api.spotify.com/v1/audio-analysis/6QeM...,277600,4,0.0
spotify:track:1TKRumlRMXv2SEKPhueya0,The Step Daddy (Explicit),6RhzdrHfuZEMnZTNRjbTpz,Hitman Sammy Sam,Step Daddy,https://open.spotify.com/track/1TKRumlRMXv2SEK...,24,2020-04-02T13:23:41Z,2003-01-01,2003,0.930,...,0.0767,0.348,118.997,audio_features,1TKRumlRMXv2SEKPhueya0,https://api.spotify.com/v1/tracks/1TKRumlRMXv2...,https://api.spotify.com/v1/audio-analysis/1TKR...,230493,4,0.0


In [6]:
print('')
print('****PART 4 of 5: Scoring Your Artists***')
genres = []
for df in [my_top_artists_genre_count_df, liked_genre_count_df, disliked_genre_count_df]:
    if df.shape[0]>0:
        genres.append(df)
genre_count_df = pd.concat(genres)
    
#my_songs_df = pd.read_pickle("./my_songs.pkl")


****PART 4 of 5: Scoring Your Artists***


In [7]:
print('****PART 5 of 5: Applying Artist ranking to featured playlist songs to create your new playlist****')
print(f"Creating featured playlist songs for: {(pd.Timestamp.now()-timedelta(days=0*365)).strftime('%Y-%m-%dT%H:%M:%S.%Z')}")
playlist_df = get_featured_playlist_uris('US',(pd.Timestamp.now()-timedelta(days=0*365)).strftime('%Y-%m-%dT%H:%M:%S.%Z'))
# print(f"{display(playlist_df)}")
raw_featured_playlist_songs_list = get_raw_featured_playlist_song_list(playlist_df)
featured_playlist_song_df = song_metadata_to_df(raw_featured_playlist_songs_list)
#featured_playlist_song_df = add_audio_feats(featured_playlist_song_df)
genre_df = get_genres(featured_playlist_song_df)
genre_exploded_df = explode_genres(genre_df)
featured_playlist_song_df = add_genres(featured_playlist_song_df, genre_exploded_df)
# featured_playlist_song_df1 = establish_genre_score(genre_count_df, featured_playlist_song_df)
featured_playlist_song_df2 = establish_genre_score(genre_count_df, featured_playlist_song_df)
# union operation with pd.concat
# featured_playlist_song_df3 = pd.concat(featured_playlist_song_df1, featured_playlist_song_df2)
create_playlist(pred_like_playlist_name, featured_playlist_song_df2)
print('****Complete! Check Spotify to see if playlist was filled with songs.****')

****PART 5 of 5: Applying Artist ranking to featured playlist songs to create your new playlist****
Creating featured playlist songs for: 2020-09-27T20:42:20.
Getting raw_featured_playlist_df


,playlist_uri,song_count
playlist,,
Roots Rising,spotify:playlist:37i9dQZF1DWYV7OOaGhoH0,100
Get Turnt,spotify:playlist:37i9dQZF1DWY4xHQp97fN6,100
Teen Beats,spotify:playlist:37i9dQZF1DWWvvyNmW9V9a,103
Rock This,spotify:playlist:37i9dQZF1DXcF6B6QPhFDv,50
Gold Edition,spotify:playlist:37i9dQZF1DWXnexX7CktaI,40
Free & Easy,spotify:playlist:37i9dQZF1DX6fhMYWIyuww,52
Feelin' Good,spotify:playlist:37i9dQZF1DX9XIFQuFvzM4,100
All New Indie,spotify:playlist:37i9dQZF1DXdbXrPNafg9d,101
Pop Remix,spotify:playlist:37i9dQZF1DXcZDD7cfEKhW,50


Transforming song raw data list into DataFrame(df) - this is basically a python version of an excel spreadsheet
Logged in as 1254636534
0/690 parsed
100/690 parsed
200/690 parsed
300/690 parsed
400/690 parsed
500/690 parsed
600/690 parsed


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year
uri,,,,,,,,,
spotify:track:53dyuOtRb9WMz5puNHodZN,Ocean Bed,3DkhgIw7lIyxekurpXNTrm,Darlingside,Ocean Bed,https://open.spotify.com/track/53dyuOtRb9WMz5p...,56,2020-09-25T04:02:00Z,2020-07-15,2020
spotify:track:4ysVx7wS2J8dvk77ON8jVI,Distant Axis,27jRNjIvlUcGN7FBRDnqhp,Matt Berninger,Distant Axis,https://open.spotify.com/track/4ysVx7wS2J8dvk7...,52,2020-09-25T04:02:00Z,2020-07-17,2020
spotify:track:1dgCF73ruK4qTXrx0TZ0j7,Berlin,7iPH2BRBF9wKa6ljxvdext,Fenne Lily,Berlin,https://open.spotify.com/track/1dgCF73ruK4qTXr...,51,2020-09-25T04:02:00Z,2020-07-21,2020
spotify:track:3pYB28IRzhtR5cHXLINchp,Shore,4EVpmkEwrLYEg6jIsiPMIb,Fleet Foxes,Can I Believe You,https://open.spotify.com/track/3pYB28IRzhtR5cH...,66,2020-09-25T04:02:00Z,2020-09-22,2020
spotify:track:1StMd0fcQFpwrkYTJuHj8z,Don't Let Me Disappear,45lorWzrKLxfKlWpV7r9CN,Ben Harper,Don't Let Me Disappear,https://open.spotify.com/track/1StMd0fcQFpwrkY...,54,2020-09-25T04:02:00Z,2020-05-13,2020


Getting genres ...
442 distinct artists
0/442 parsed
0:Darlingside:['boston folk', 'folk-pop', 'indie folk', 'new americana', 'new england americana', 'stomp and holler']
1:Matt Berninger:['indie rock']
2:Fenne Lily:['bristol indie', 'british singer-songwriter', 'chamber pop', 'indie anthem-folk', 'indie folk']
3:Fleet Foxes:['chamber pop', 'indie folk', 'indie pop', 'indie rock', 'seattle indie', 'stomp and holler']
4:Ben Harper:['neo mellow']
5:Belle Mt.:['indie anthem-folk']
6:Half Moon Run:['canadian indie', 'indie quebecois', 'quebec indie']
7:The Lumineers:['folk-pop', 'modern rock', 'stomp and holler']
8:Mt. Joy:['indie folk', 'indie rock', 'modern rock', 'stomp and holler']
9:Thomas Csorba:['deep new americana']
10:Lera Lynn:['alternative country', 'new americana']
11:Ray LaMontagne:['acoustic pop', 'indie folk', 'neo mellow', 'new americana', 'pop rock']
12:The War and Treaty:['funk']
13:Sarah Jarosz:['folk', 'indie folk', 'lilith', 'new americana', 'progressive bluegrass', 's

...

Logged in as 1254636534


,playlist_uri,song_count
playlist,,
FreshJams @2020-09-27 20:40:02.535628,spotify:playlist:6WYEZTnRJlQD3326wIuZDn,0
Jams 9/27 12:00,spotify:playlist:79EHB83wBhmDy5hdiess73,29
"Jams 9/26, 9:40pm",spotify:playlist:1g2dqbzbKfn4I7RfggDGHW,28
"Jams 9/26, 8:35pm",spotify:playlist:6WFIRi0vyvxn8ZGzndekEa,25
Dislikes,spotify:playlist:4zoDh60COnrbyctm9z9h77,454
My Shazam Tracks,spotify:playlist:5Gy5xebfLSaOms4klT0NAD,50
Discover Weekly,spotify:playlist:37i9dQZEVXcPcrlohfCkeD,30


,playlist_uri,song_count
playlist,,
FreshJams @2020-09-27 20:40:02.535628,spotify:playlist:6WYEZTnRJlQD3326wIuZDn,0
Jams 9/27 12:00,spotify:playlist:79EHB83wBhmDy5hdiess73,29
"Jams 9/26, 9:40pm",spotify:playlist:1g2dqbzbKfn4I7RfggDGHW,28
"Jams 9/26, 8:35pm",spotify:playlist:6WFIRi0vyvxn8ZGzndekEa,25
Dislikes,spotify:playlist:4zoDh60COnrbyctm9z9h77,454
My Shazam Tracks,spotify:playlist:5Gy5xebfLSaOms4klT0NAD,50
Discover Weekly,spotify:playlist:37i9dQZEVXcPcrlohfCkeD,30


,album,artist_uri,artist,track,url,popularity,added_at,release_date,release_year,genre score,pct_rank
uri,,,,,,,,,,,
spotify:track:14BcEAdv3OkOXYjGIk7jTZ,Statues in the Garden (Arras),75dQReiBOHN37fQgWQrIAJ,Local Natives,Statues in the Garden (Arras),https://open.spotify.com/track/14BcEAdv3OkOXYj...,58,2020-09-18T04:01:00Z,2020-09-17,2020,892.0,1.000000
spotify:track:1zNrtAQhwbRyFmsf3yoBqT,The Great Divide,4LG4Bs1Gadht7TCrMytQUO,The Shins,The Great Divide,https://open.spotify.com/track/1zNrtAQhwbRyFms...,49,2020-09-25T04:01:00Z,2020-09-24,2020,744.0,0.998148
spotify:track:51309AlWfhaco8OwvUWlZI,Mr. Lonely,6mKqFxGMS5TGDZI3XkT5Rt,Angel Olsen,Mr. Lonely,https://open.spotify.com/track/51309AlWfhaco8O...,52,2020-09-18T04:01:00Z,2020-09-16,2020,728.0,0.996296
spotify:track:3pYB28IRzhtR5cHXLINchp,Shore,4EVpmkEwrLYEg6jIsiPMIb,Fleet Foxes,Can I Believe You,https://open.spotify.com/track/3pYB28IRzhtR5cH...,66,2020-09-25T04:01:00Z,2020-09-22,2020,636.0,0.994444
spotify:track:4hyAvXnUZwgOSDGf59fYTG,Host,3Oim8XBPbznAa8Jj8QzNc8,Cults,A Low,https://open.spotify.com/track/4hyAvXnUZwgOSDG...,47,2020-09-18T04:01:00Z,2020-09-18,2020,590.0,0.992593


****Complete! Check Spotify to see if playlist was filled with songs.****


In [ ]:
################## END OF OFFICIAL SCRIPT - BELOW IS JUST PLAYING AROUND ###############

In [ ]:

display(featured_playlist_song_df2.head())